# Tools

## Github

#### Clone a repository based on url

In [ ]:
from typing import Optional, Dict, Any, List
from logging import Handler

class ArchLensConfig:
  def __init__(
    self,
    name: str,
    rootFolder: str,
    views: Dict[str, Dict[str, List[Dict[str, Any]]]]
  ):
    self.name = name
    self.rootFolder = rootFolder
    self.saveLocation = "./diagrams/"
    self.views = views

  @staticmethod
  def from_dict(data: Dict[str, Any]) -> 'ArchLensConfig':
    github = data['github']
    views = {}
    for view_name, view_data in data['views'].items():
      views[view_name] = view_data['packages']
    return ArchLensConfig(
      schema=data.get('$schema', ''),
      name=data.get('name', ''),
      root_folder=data.get('rootFolder', ''),
      github_url=github.get('url', ''),
      github_branch=github.get('branch', ''),
      save_location=data.get('saveLocation', ''),
      views=views
    )
  
#create Archlensobject
viewsJson = {"top-level-view-depth-1": {
      "packages": [
        {
          "path": "*",
          "depth": 1
        }
      ]
    },
    "top-level-view-depth-2": {
      "packages": [
        {
          "path": "*",
          "depth": 2
        }
      ]
    }}

archlensObject = ArchLensConfig(name='testing' ,rootFolder='zeeguu', views=viewsJson)

print(archlensObject)



In [3]:
from typing import Optional, Dict, Any
from git import Repo, GitCommandError
from langchain.tools import tool
import os, shutil


@tool("git_clone")
def git_clone_tool(
    repo_url: str,
    dest: str,
    branch: Optional[str] = None,
    overwrite: bool = False,
) -> Dict[str, Any]:
    """
    Clone a Git repository into ./repositories/{dest} using GitPython.

    Args:
        repo_url: HTTPS or SSH URL of the repository.
        dest: Name of the destination folder for the clone inside ./repositories/.
        branch: Optional branch to check out.
        overwrite: If True, overwrite existing destination folder.
    Returns:
        A dict with success (bool), dest (str), and error/stdout messages.
    """

    try:
        # Ensure repositories/ root exists
        root_dir = os.path.join(os.getcwd(), "repositories")
        os.makedirs(root_dir, exist_ok=True)

        # Full destination path inside repositories/
        full_dest = os.path.join(root_dir, dest)

        # Handle overwrite
        if os.path.exists(full_dest):
            if overwrite:
                shutil.rmtree(full_dest)
            else:
                return {"success": False, "error": f"Destination {full_dest} already exists."}


        # Clone options
        kwargs = {}
        if branch:
            kwargs["branch"] = branch
            full_dest = f"{full_dest}/{branch}"

        os.makedirs(full_dest, exist_ok=True)
        repo = Repo.clone_from(repo_url, full_dest, **kwargs)

        return {
            "success": True,
            "dest": full_dest,
            "branch": repo.active_branch.name if not repo.head.is_detached else "detached",
            "error": None,
        }
    except GitCommandError as e:
        return {"success": False, "dest": dest, "error": str(e)}
    except Exception as e:
        return {"success": False, "dest": dest, "error": str(e)}

In [ ]:
import json

@tool('init_archLens')
def init_archLens(repo_url: str):
    """"If you have cloned the arch-reconstruct-ai repository, initialize archLens."""

    repo_name = repo_url.rstrip('/').split('/')[-1]
    if repo_name.endswith('.git'):
        repo_name = repo_name[:-4]
    print(f"Repository name: {repo_name}")

    repo_path = "/Users/nikolajworsoelarsen/Desktop/CSKandidat/Thesis/arch-reconstruct-ai/experiments/tool-calling/repositories/"+repo_name
    os.chdir(repo_path)

    if os.path.exists("archlens.json"):
        print("archlens.json already exists, skipping initialization.")
    else:
        os.system("archlens init")

    return "Initialized archLens"

@tool('run_archLens')
def run_archLens(repo_url: str):
    """"Run archLens on the cloned repository."""

    repo_name = repo_url.rstrip('/').split('/')[-1]
    if repo_name.endswith('.git'):
        repo_name = repo_name[:-4]
    print(f"Repository name: {repo_name}")
    
    repo_path = "/Users/nikolajworsoelarsen/Desktop/CSKandidat/Thesis/arch-reconstruct-ai/experiments/tool-calling/repositories/"+repo_name
    os.chdir(repo_path)

    if not os.path.exists("archlens.json"):
        return "archlens.json does not exist. Please run init_archLens first."

    os.system(f"archlens render")

    return "Ran archLens"


@tool('read_archLens_config_file')
def read_archLens_config_file(repo_url: str):
    """"Reads the content of the archlens.json file."""

    repo_name = repo_url.rstrip('/').split('/')[-1]
    if repo_name.endswith('.git'):
        repo_name = repo_name[:-4]
    print(f"Repository name: {repo_name}")

    repo_path = "/Users/nikolajworsoelarsen/Desktop/CSKandidat/Thesis/arch-reconstruct-ai/experiments/tool-calling/repositories/"+repo_name
    os.chdir(repo_path)

    config_path = "archlens.json"
    if not os.path.exists(config_path):
        return "archlens.json does not exist. Please run init_archLens first."
    
    with open('archlens.json', 'r') as file:
        data = json.load(file)

    print(json.dumps(data, indent=4))

    return "Read config file"


@tool('write_archLens_config_file')
def write_archLens_config_file(repo_url: str, arch: ArchLensConfig):
    """"Writes content to the archlens.json file.
        args: 
    """
    repo_name = repo_url.rstrip('/').split('/')[-1]
    if repo_name.endswith('.git'):
        repo_name = repo_name[:-4]
    print(f"Repository name: {repo_name}")

    repo_path = "/Users/nikolajworsoelarsen/Desktop/CSKandidat/Thesis/arch-reconstruct-ai/experiments/tool-calling/repositories/"+repo_name
    os.chdir(repo_path)

    config_path = "archlens.json"
    with open(config_path, 'w') as file:
        json.dump(arch.__dict__, file)
    
    return "Wrote to config file"

write_archLens_config_file({"repo_url": "https://github.com/zeeguu/api.git", "arch": archlensObject})

#doesn't work yet
run_archLens('https://github.com/zeeguu/api.git')

Repository name: api
Repository name: api
analyzing /Users/nikolajworsoelarsen/Desktop/CSKandidat/Thesis/arch-reconstruct-ai/experiments/tool-calling/repositories/api/zeeguu
analyzing /Users/nikolajworsoelarsen/Desktop/CSKandidat/Thesis/arch-reconstruct-ai/experiments/tool-calling/repositories/api/zeeguu/core
analyzing /Users/nikolajworsoelarsen/Desktop/CSKandidat/Thesis/arch-reconstruct-ai/experiments/tool-calling/repositories/api/zeeguu/core/user_activity_hooks
analyzing /Users/nikolajworsoelarsen/Desktop/CSKandidat/Thesis/arch-reconstruct-ai/experiments/tool-calling/repositories/api/zeeguu/core/account_management
analyzing /Users/nikolajworsoelarsen/Desktop/CSKandidat/Thesis/arch-reconstruct-ai/experiments/tool-calling/repositories/api/zeeguu/core/word_filter
analyzing /Users/nikolajworsoelarsen/Desktop/CSKandidat/Thesis/arch-reconstruct-ai/experiments/tool-calling/repositories/api/zeeguu/core/word_scheduling
analyzing /Users/nikolajworsoelarsen/Desktop/CSKandidat/Thesis/arch-recons

'Ran archLens'

In [35]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage

agent = create_agent(
    "openai:gpt-5-nano",
    tools=[git_clone_tool, init_archLens, run_archLens, read_archLens_config_file],
    prompt="Act as an assistant, that does architectural reconstruction of software repositories. Archlens is a tool which can be used for python repositories",
)

#result = agent.invoke({"messages": [HumanMessage("Can you clone the following github repository: https://github.com/simonskodt/arch-reconstruct-ai, feel free to overwrite if a clone already exists, ")]})
result = agent.invoke({"messages": [HumanMessage("Can you read the content of the archlens configuration file of https://github.com/zeeguu/api.git")]})
print(result.get("messages"))

Repository name: api
{
    "$schema": "https://raw.githubusercontent.com/archlens/ArchLens/master/src/config.schema.json",
    "name": "-",
    "rootFolder": "zeeguu",
    "github": {
        "url": "https://github.com/zeeguu/api",
        "branch": "master"
    },
    "saveLocation": "./diagrams/",
    "views": {
        "top-level-view-depth-1": {
            "packages": [
                {
                    "path": "*",
                    "depth": 1
                }
            ]
        }
    }
}
Repository name: api
{
    "$schema": "https://raw.githubusercontent.com/archlens/ArchLens/master/src/config.schema.json",
    "name": "-",
    "rootFolder": "zeeguu",
    "github": {
        "url": "https://github.com/zeeguu/api",
        "branch": "master"
    },
    "saveLocation": "./diagrams/",
    "views": {
        "top-level-view-depth-1": {
            "packages": [
                {
                    "path": "*",
                    "depth": 1
                }
            

#### Add Github MCP server based on url

In [3]:
from typing import Dict, Any
from langchain.tools import tool
from experiments.utils.mcp_client_factory import load_mcp_config, save_mcp_config, create_mcp_client_from_config


@tool("add_mcp_server")
def add_mcp_server_tool(
    name: str,
    url: str,
    transport: str = "streamable_http",
) -> Dict[str, Any]:
    """
    Add a new MCP server to the configuration.
    
    Args:
        name: Name identifier for the MCP server
        url: URL of the MCP server
        transport: Transport type (default: "streamable_http")
    Returns:
        Dict with success status and current config

    Note: 
        MCP tools accessed via clients are not hot reloaded or dynamically  updated,
        a new agent or tool instance has to be  
    """
    try:
        # Load existing config
        config = load_mcp_config()
        
        # Add new server
        config[name] = {
            "url": url,
            "transport": transport
        }
    
        save_mcp_config(config)
        
        return {
            "success": True,
            "message": f"Added MCP server '{name}'",
            "config": config
        }
    except Exception as e:
        return {
            "success": False,
            "error": str(e),
            "config": {}
        }
    
@tool("add_github_repository_as_mcp_server")
def add_github_repository_as_mcp_tool(repo_url: str, server_name: str) -> Dict[str, Any]:
    """
    Add a GitHub repository as an MCP server using gitmcp.io.
    
    Args:
        repo_url: GitHub repository URL (e.g., https://github.com/owner/repo)
        server_name: Name identifier for the MCP server
    Returns:
        Dict with success status and current config
    """
    # Extract the repository path from the GitHub URL
    if "github.com/" in repo_url:
        # Extract everything after github.com/
        repo_path = repo_url.split("github.com/", 1)[1]
        # Remove .git suffix if present
        if repo_path.endswith(".git"):
            repo_path = repo_path[:-4]
        gitmcp_url = f"https://gitmcp.io/{repo_path}"
    else:
        raise ValueError("Invalid GitHub repository URL")
    
    tool_input = {
        "name": server_name,
        "url": gitmcp_url,
    }

    return add_mcp_server_tool.invoke(tool_input)

    

@tool("remove_mcp_server")
def remove_mcp_server_tool(
    name: str,
) -> Dict[str, Any]:
    """
    Remove an MCP server from the configuration.
    
    Args:
        name: Name identifier of the MCP server to remove
    Returns:
        Dict with success status and current config

    Note: 
        MCP tools accessed via clients are not hot reloaded or dynamically  updated,
        a new agent or tool instance has to be 
    """
    try:
        config = load_mcp_config()
        
        if name not in config:
            return {
                "success": False,
                "error": f"MCP server '{name}' not found",
                "config": config
            }
        
        del config[name]
        save_mcp_config(config)
        
        return {
            "success": True,
            "message": f"Removed MCP server '{name}'",
            "config": config
        }
    except Exception as e:
        return {
            "success": False,
            "error": str(e),
            "config": {}
        }

@tool("list_mcp_servers")
def list_mcp_servers() -> Dict[str, Any]:
    """
    List all configured MCP servers.
    
    Returns:
        Dict with success status and list of servers
    """
    try:
        config = load_mcp_config()
        return {
            "success": True,
            "servers": list(config.keys()),
            "config": config
        }
    except Exception as e:
        return {
            "success": False,
            "error": str(e),
            "config": {}
        }

ModuleNotFoundError: No module named 'experiments'

In [ ]:
from langchain_core.messages import HumanMessage

import sys
import os
# Add the parent directory to sys.path so 'experiments' can be imported
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))
from experiments.utils.agent_factory import create_agent_with_valid_tools

tools = [add_github_repository_as_mcp_tool, add_mcp_server_tool, remove_mcp_server_tool, list_mcp_servers, git_clone_tool]

agent = create_agent_with_valid_tools(
    "openai:gpt-5-nano",
    tools=tools,
    prompt="""Act as an assistant.
                When using tools:
                - Use tools if relevant before answering.
            """
)


stream = agent.astream({"messages": [HumanMessage("Can you clone the github repository: https://github.com/simonskodt/arch-reconstruct-ai")]})
async for chunk in stream:
    print(chunk)


result = await agent.ainvoke({"messages": [HumanMessage("Can you list the MCP servers available")]})
print(result)

result = await agent.ainvoke({"messages": [HumanMessage("Can you take the following github repository: https://github.com/simonskodt/arch-reconstruct-ai, and make it into a MCP server")]})
print(result)




client = create_mcp_client_from_config()
mcp_tools = await client.get_tools() 
tools += mcp_tools

[print(tool.name) for tool in tools]

agent = create_agent_with_valid_tools(
    "openai:gpt-5-nano",
    tools=tools, # Tools cannot be dynamically  or hot reloaded?, agent has to be recreated  
    prompt="""Act as an assistant.
                When using tools:
                - Use tools if relevant before answering.
            """
)
result = await agent.ainvoke({"messages": [HumanMessage("Can you list the MCP servers available")]})
print(result)



{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_iqNmvENZzhZ7Wh3kntQPXqkJ', 'function': {'arguments': '{"repo_url":"https://github.com/simonskodt/arch-reconstruct-ai","dest":"arch-reconstruct-ai","branch":null,"overwrite":false}', 'name': 'git_clone'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 308, 'prompt_tokens': 511, 'total_tokens': 819, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CLo4ayGmdBLSAkgAUugCxyrMtnlJ7', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--277e2183-d669-49fd-844c-d2a1f3f364b6-0', tool_calls=[{'name': 'git_clone', 'args': {'repo_url': 'https://github.com/simonskodt/arch-reconstruct-ai', '